<a href="https://colab.research.google.com/github/fatjan/learn-tensorflow/blob/master/Ex4_Happy_or_Sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2020-12-12 06:04:24--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.240, 142.250.31.128, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-12-12 06:04:25 (230 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [13]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
])

from tensorflow.keras.optimizers import RMSprop

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 16)        4624      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 16)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4624)             

In [9]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size = (150, 150),
        batch_size = 10,
        class_mode = 'binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [14]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
        train_generator,
        steps_per_epoch = 8,
        epochs=15,
        verbose = 1,
        callbacks = [callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 0s 16ms/step - loss: 0.9094 - accuracy: 0.5250
Epoch 2/15
8/8 [==============================] - 0s 17ms/step - loss: 0.7086 - accuracy: 0.8000
Epoch 3/15
8/8 [==============================] - 0s 17ms/step - loss: 0.2936 - accuracy: 0.9000
Epoch 4/15
8/8 [==============================] - 0s 17ms/step - loss: 0.2001 - accuracy: 0.9500
Epoch 5/15
8/8 [==============================] - 0s 17ms/step - loss: 0.1549 - accuracy: 0.9375
Epoch 6/15
8/8 [==============================] - 0s 17ms/step - loss: 0.0817 - accuracy: 0.9625
Epoch 7/15
8/8 [==============================] - 0s 17ms/step - loss: 0.1136 - accuracy: 0.9625
Epoch 8/15
8/8 [==============================] - 0s 19ms/step - loss: 0.1199 - accuracy: 0.9250
Epoch 9/15
8/8 [==============================] - 0s 17ms/step - loss: 0.0597 - accuracy: 0.9750
Epoch 10/15
8/8 [==============================] - 0s 17ms/step - loss: 0.0435 - accuracy: 0.9875
Epoch 11/15
7/8 [============